In [1]:
!nvidia-smi

Sat Dec  9 14:14:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Downloading Images

In [2]:
# install bing images downloader package
!pip install bing-image-downloader

In [3]:
!mkdir data

In [4]:
from bing_image_downloader import downloader
common_cat_breeds = ['Maine Coon', 'Persian Cat', 'American Shorthair', 'Siamese Cat'] #
for cat in common_cat_breeds:
  downloader.download(cat, limit=350,  output_dir='data', timeout=60)

[%] Downloading Images to /content/data/Maine Coon


[!!]Indexing page: 1

[%] Indexed 35 Images on Page 1.


[%] Downloading Image #1 from https://mainecoonhawaii.com/wp-content/uploads/2020/06/Ludo-rsz-1.jpg
[!] Issue getting: https://mainecoonhawaii.com/wp-content/uploads/2020/06/Ludo-rsz-1.jpg
[!] Error:: Remote end closed connection without response
[%] Downloading Image #1 from https://imagesvc.meredithcorp.io/v3/mm/image?url=https:%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F47%2F2020%2F08%2F16%2Fmaine-coon-992070132-2000.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://4.bp.blogspot.com/-kLjgbxbdd3M/Wk_uRD6-UUI/AAAAAAAAHc0/1KHrzTYs-QsbDjEZRw1AU6XturrQKuMRwCLcBGAs/s1600/maine-coon-2.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://www.shared.com/content/images/2017/10/maine-coon-cat-photography-robert-sijka-55-57ad8f1b7ddd2__880.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://lifeasahuman.com/files/2020/03/Main-Coon-2-s

In [5]:
# Clean data
import os
import cv2
from PIL import Image
import numpy as np
import random

folders = common_cat_breeds

for folder in folders:
    print('Cleaning folder: ', folder)
    for i in os.listdir(f"data/{folder}"):
        img = cv2.imread(f"data/{folder}/{i}")
        if img is None or (not i.endswith('.jpg') and not i.endswith('.jpeg') and not i.endswith('.png')):
            print('img is none: ', i)
            os.system(f"rm 'data/{folder}/{i}'")

    for i in os.listdir(f"data/{folder}"):
        img = np.array(Image.open(f"data/{folder}/{i}").convert('RGB'))
        cv2.imwrite(f"data/{folder}/{i}", img)


Cleaning folder:  Maine Coon
img is none:  Image_9.JPG
Cleaning folder:  Persian Cat
Cleaning folder:  American Shorthair
Cleaning folder:  Siamese Cat


In [6]:
# Split data into train-val and test
import random

random.seed(42)
os.mkdir('data/train')
os.mkdir('data/test')

for folder in folders:
	files = os.listdir(f'data/{folder}')
	random.shuffle(files)
	t = int(0.8 * len(files))
	os.system(f"mv 'data/{folder}' data/train/")
	os.system(f"mkdir 'data/test/{folder}'")
	for f in files[t:]:
		os.system(f"mv 'data/train/{folder}/{f}' 'data/test/{folder}/'")

# Training the Model

In [7]:
## Importing the libraries
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import sklearn.metrics as metrics
import seaborn as sns


In [8]:
## Hyperparams ##

LEARNING_RATE = 0.0001 # learning rate to be used by the model
BATCH_SIZE = 4
EPOCHS = 5
CLASSES = ['Maine Coon', 'Persian Cat', 'American Shorthair', 'Siamese Cat'] #
NUM_CLASSES = len(CLASSES)
IMG_SIZE = (224, 224) #

In [9]:

"""
Function: build_model
Description: Builds the model by using the pretrained model and adding the dropout layer and the dense layer
"""
def build_model(NUM_CLASSES, LEARNING_RATE):

    ## Input Layer ##

    input_layer = keras.layers.Input(shape=(224, 224, 3))

    ## EfficientNet B0 pretrain
    model = hub.KerasLayer('https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1', trainable=False)

    ## Dropout layer
    dropout = keras.layers.Dropout(rate=0.2)

    ## Dense layer
    dense = keras.layers.Dense(NUM_CLASSES, activation='softmax')

    ## Build model by adding the pretrained model and the new layers
    model = keras.Sequential([
        input_layer,
        model,
        dropout,
        dense
    ])

    ## Compile model
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [10]:
model = build_model(NUM_CLASSES, LEARNING_RATE)

In [11]:

"""
Function : load_images
Description: Loads the images from the directory

"""
def load_dataset(directory, batch_size, image_size):

    ## ImageDataGenerator is used to perform data augmentation
    ## We use the following augmentation techniques to increase dataset size:
    ## - Horizontal flip: Flip the image horizontally, ie left to right
    ## - Rotate: Rotate the image by a random angle between -20 and 20 degrees
    ## - Zoom: Zoom in or out by a random factor between 0.8 and 1.2 times
    ## - Shift: Shift the image by a random horizontal and vertical offset of 20% of the image width or height
    train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2,
        horizontal_flip=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest'
    )

    train_dataset = train_data_gen.flow_from_directory(
        directory,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )

    validation_dataset = train_data_gen.flow_from_directory(
        directory,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )

    return train_dataset, validation_dataset

In [12]:
## Load the dataset
train_dataset, validation_dataset = load_dataset('data/train', BATCH_SIZE, IMG_SIZE)

Found 68 images belonging to 4 classes.
Found 16 images belonging to 4 classes.


In [13]:
"""
Function: train
Description: Trains the model, saves the model and plots the training and validation loss and accuracy

"""

def train(model, train_dataset, validation_dataset, callbacks, epochs):

    history = model.fit_generator(
        train_dataset,
        epochs=epochs,
        validation_data=validation_dataset,
        verbose=1,
        callbacks=callbacks
    )

    return history

In [14]:
### Callbacks that will be called while training the model
## - ModelCheckpoint: Saves the model after every epoch
## - TensorBoard: Visualize the training and validation on browser

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath='models/model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

In [15]:
# Finally train the model, should see a progress bar while training
history = train(model, train_dataset, validation_dataset, callbacks=callbacks, epochs=EPOCHS)

<ipython-input-13-640e4bfc1ee9>:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/5
17/17 [==============================] - ETA: 0s - loss: 1.4095 - accuracy: 0.2353

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


17/17 [==============================] - 22s 181ms/step - loss: 1.4095 - accuracy: 0.2353 - val_loss: 1.3192 - val_accuracy: 0.2500
Epoch 2/5
17/17 [==============================] - 2s 124ms/step - loss: 1.2710 - accuracy: 0.3676 - val_loss: 1.0842 - val_accuracy: 0.5000
Epoch 3/5
17/17 [==============================] - 2s 132ms/step - loss: 1.1369 - accuracy: 0.5147 - val_loss: 1.0802 - val_accuracy: 0.5625
Epoch 4/5
17/17 [==============================] - 3s 192ms/step - loss: 1.0737 - accuracy: 0.5147 - val_loss: 1.0454 - val_accuracy: 0.5625
Epoch 5/5
17/17 [==============================] - 2s 125ms/step - loss: 0.9842 - accuracy: 0.6324 - val_loss: 0.9317 - val_accuracy: 0.6875


In [ ]:
## Plot the training and validation loss and accuracy
fig = plt.figure(figsize=(10, 10))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('loss.png')

fig = plt.figure(figsize=(10, 10))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('accuracy.png')

# Testing the Model

In [ ]:
# Getting the test dataset
test_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.0,
    fill_mode='nearest',
)

test_dataset = test_data_gen.flow_from_directory(
    'data/test',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
## Do predictions on the test set
predictions = model.predict(test_dataset)

In [ ]:
## Get the original labels
test_labels = test_dataset.labels

In [ ]:
## Compute confusion matrix
confusion = metrics.confusion_matrix(test_labels, predictions.argmax(1)).astype('float32')
confusion = confusion.astype("float") / confusion.sum(axis=1)

In [ ]:
## Get heatmap
fig = plt.figure(figsize=(10, 10))
ax = sns.heatmap(
    confusion, xticklabels=CLASSES, yticklabels=CLASSES,
    cmap='Blues')

ax.figure.subplots_adjust(left = 0.3, bottom=0.4)


plt.title("Confusion matrix")
plt.ylabel("True label")
plt.xlabel("Predicted label")

plt.savefig('test_confusion_matrix.png')